#Deep Networks

#1. CNN

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Step 2: Load and prepare the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

# Step 4: Define the neural network architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 5: Define loss function and optimizer
net = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Step 6: Training loop
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 200}')
            running_loss = 0.0

print('Finished Training')

100%|██████████| 170498071/170498071 [00:01<00:00, 96430040.04it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, Batch 200, Loss: 1.8168666952848433
Epoch 1, Batch 400, Loss: 1.475939399600029
Epoch 1, Batch 600, Loss: 1.437886941730976
Epoch 1, Batch 800, Loss: 1.3338669866323472
Epoch 1, Batch 1000, Loss: 1.3135102832317351
Epoch 1, Batch 1200, Loss: 1.264631651043892
Epoch 1, Batch 1400, Loss: 1.2289302706718446


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 2, Batch 200, Loss: 1.1367645198106766
Epoch 2, Batch 400, Loss: 1.1263438805937767
Epoch 2, Batch 600, Loss: 1.1252578487992286
Epoch 2, Batch 800, Loss: 1.10054334461689
Epoch 2, Batch 1000, Loss: 1.0812892356514932
Epoch 2, Batch 1200, Loss: 1.079770558178425
Epoch 2, Batch 1400, Loss: 1.075558524429798
Epoch 3, Batch 200, Loss: 0.9857586613297462
Epoch 3, Batch 400, Loss: 0.9978375771641731
Epoch 3, Batch 600, Loss: 0.9597646543383598
Epoch 3, Batch 800, Loss: 0.9696293148398399
Epoch 3, Batch 1000, Loss: 0.9665106198191643
Epoch 3, Batch 1200, Loss: 0.9465541070699692
Epoch 3, Batch 1400, Loss: 0.9593505921959877
Epoch 4, Batch 200, Loss: 0.8755633534491062
Epoch 4, Batch 400, Loss: 0.8804466012120247
Epoch 4, Batch 600, Loss: 0.8861808833479882
Epoch 4, Batch 800, Loss: 0.8768923822045326
Epoch 4, Batch 1000, Loss: 0.8654893162846565
Epoch 4, Batch 1200, Loss: 0.8860401554405689
Epoch 4, Batch 1400, Loss: 0.8773477427661419
Epoch 5, Batch 200, Loss: 0.7842339450120925
Epoch

#2. RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Generate synthetic dataset
def generate_sequence(seq_length=20):
    return np.random.randint(0, 100, size=(seq_length))

# Create input-output pairs for training
def create_dataset(seq_length=20, num_samples=1000):
    X, y = [], []
    for _ in range(num_samples):
        sequence = generate_sequence(seq_length)
        X.append(sequence[:-1])  # Input sequence (all but the last element)
        y.append(sequence[1:])   # Output sequence (all but the first element)
    return torch.tensor(X).float(), torch.tensor(y).float()

# Define the Recurrent Neural Network (RNN) model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # Forward pass through the RNN layer
        out, hidden = self.rnn(x, hidden)
        # Flatten the output for the fully connected layer
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

# Create training dataset
input_size = 1  # Dimensionality of each item in the sequence
hidden_size = 64  # Number of features in the RNN hidden state
output_size = 1  # Dimensionality of each predicted item in the sequence
seq_length = 20  # Length of input sequence
num_samples = 1000  # Number of training samples

X_train, y_train = create_dataset(seq_length, num_samples)

# Define model, loss function, and optimizer
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i in range(0, num_samples, batch_size):
        # Prepare batch
        inputs = X_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        # Forward pass
        optimizer.zero_grad()
        hidden = model.init_hidden(len(inputs))
        outputs, _ = model(inputs.unsqueeze(-1), hidden)

        # Compute loss
        loss = criterion(outputs, targets.view(-1))

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {epoch_loss / (num_samples / batch_size)}')

print('Finished Training')

<ipython-input-2-9948cdc7bc70>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(X).float(), torch.tensor(y).float()
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([608])) that is different to the input size (torch.Size([608, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([152])) that is different to the input size (torch.Size([152, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_l

Epoch 1, Loss: 3104.47771875
Epoch 2, Loss: 2865.2366171875
Epoch 3, Loss: 2666.5357890625
Epoch 4, Loss: 2488.11859375
Epoch 5, Loss: 2324.5311015625
Epoch 6, Loss: 2176.04287890625
Epoch 7, Loss: 2040.529984375
Epoch 8, Loss: 1916.65119140625
Epoch 9, Loss: 1803.44687890625
Epoch 10, Loss: 1700.12764453125
Epoch 11, Loss: 1606.000578125
Epoch 12, Loss: 1520.434734375
Epoch 13, Loss: 1442.8423984375
Epoch 14, Loss: 1372.66919140625
Epoch 15, Loss: 1309.38787890625
Epoch 16, Loss: 1252.49518359375
Epoch 17, Loss: 1201.5095859375
Epoch 18, Loss: 1155.9705859375
Epoch 19, Loss: 1115.437892578125
Epoch 20, Loss: 1079.4917890625
Epoch 21, Loss: 1047.732580078125
Epoch 22, Loss: 1019.781291015625
Epoch 23, Loss: 995.279630859375
Epoch 24, Loss: 973.890154296875
Epoch 25, Loss: 955.296517578125
Epoch 26, Loss: 939.203447265625
Epoch 27, Loss: 925.3365703125
Epoch 28, Loss: 913.4423203125
Epoch 29, Loss: 903.2875703125
Epoch 30, Loss: 894.658998046875
Epoch 31, Loss: 887.3627109375
Epoch 32, 

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Hyperparameters
sequence_length = 28  # Length of sequence (number of rows in an image)
input_size = 28  # Number of features in each time step (number of columns in an image)
hidden_size = 128  # Number of features in the hidden state
num_layers = 2  # Number of RNN layers
num_classes = 10  # Number of output classes (digits 0-9)
batch_size = 100  # Number of samples in each batch
num_epochs = 5  # Number of epochs to train the model
learning_rate = 0.001  # Learning rate

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 33657539.48it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 894346.78it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9033437.56it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2190975.13it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the model
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Epoch [1/5], Step [100/600], Loss: 1.2772
Epoch [1/5], Step [200/600], Loss: 0.5770
Epoch [1/5], Step [300/600], Loss: 0.5421
Epoch [1/5], Step [400/600], Loss: 0.5290
Epoch [1/5], Step [500/600], Loss: 0.3941
Epoch [1/5], Step [600/600], Loss: 0.3185
Epoch [2/5], Step [100/600], Loss: 0.3736
Epoch [2/5], Step [200/600], Loss: 0.3762
Epoch [2/5], Step [300/600], Loss: 0.1183
Epoch [2/5], Step [400/600], Loss: 0.1322
Epoch [2/5], Step [500/600], Loss: 0.2168
Epoch [2/5], Step [600/600], Loss: 0.1995
Epoch [3/5], Step [100/600], Loss: 0.2650
Epoch [3/5], Step [200/600], Loss: 0.1380
Epoch [3/5], Step [300/600], Loss: 0.2775
Epoch [3/5], Step [400/600], Loss: 0.2308
Epoch [3/5], Step [500/600], Loss: 0.1507
Epoch [3/5], Step [600/600], Loss: 0.0393
Epoch [4/5], Step [100/600], Loss: 0.0893
Epoch [4/5], Step [200/600], Loss: 0.1023
Epoch [4/5], Step [300/600], Loss: 0.1043
Epoch [4/5], Step [400/600], Loss: 0.2227
Epoch [4/5], Step [500/600], Loss: 0.0887
Epoch [4/5], Step [600/600], Loss: